### Set your directories.

Edit ```basedir``` according to the location of folder containing this repo.



In [ ]:
basedir = '/home/abhinavgupta0110/generalized_nCM'

import os

is_google_colab = False
is_use_GPU = False

Edit ```os.chdir(os.path.join(basedir, 'Poly_Collocation_Solver'))``` with the folder name containing this repo.

In [ ]:
if is_use_GPU:
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('No GPU found!')
    else:
        print(gpu_info)

if is_google_colab:
    from google.colab import drive
    drive.mount('/content/drive')

os.chdir(os.path.join(basedir, 'generalized_nCMs'))

%load_ext autoreload

### Load packages as per need.

In [ ]:
%autoreload 2

from src.utilities.DDE_Solver import ddeinttf 
from src.utilities.helper_classes import * 
import src.burgers_eqn_case_findiff.burgers_eqn as burg
from src.burgers_eqn_case_findiff.burgers_eqn_helper_classes import * 

from IPython.core.debugger import set_trace

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib import cm

from matplotlib.animation import FuncAnimation, PillowWriter
from numpy import random

### Define some useful classes

In [ ]:
class ad_eqn_analy:

    def __init__(self, x, app):
        self.x = x
        self.app = app

    def __call__(self, t):
        u = self.x / (t + 1.) / (1. + np.sqrt(np.divide(t + 1., self.app.t0), dtype = np.float64) * np.exp(self.app.Re * (self.x**2 / ((4. * t) + 4.)), dtype = np.float64))
        return tf.convert_to_tensor([u], dtype=tf.float32)

### Main part starts here

#### User-defined parameters and create objects

In [ ]:
args = burg.burg_args(T = 5., nt = 500, x_left = 0, x_right = 1.0, Nx = 150, Re = 250, dbc_l = 1, nbc_l = 0, dbc_r = 0, nbc_r = 1, max_deriv = 2, acc = 4, acc_advec = 4)
model_dir = 'burgers_fd/model_dir_case_test'

In [ ]:
os.chdir(basedir)

if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
os.chdir(model_dir)

In [ ]:
grid_obj = fdgen.grid(args)
u0 = initial_cond(grid_obj.x_grid, args)

t = tf.linspace(0., args.T, args.nt)

In [ ]:
deriv_obj = fdgen.deriv(args, grid_obj)

sol_obj = burg.burg_rhs(args, deriv_obj, grid_obj)

#### Solve and plot

In [ ]:
u = ddeinttf(sol_obj, u0, t, alg_name = 'vode', nsteps = 100)

In [ ]:
import pickle

data_dict = {'u':u, 't': t}

with open('burgers_T'+str(args.T)+'_nt'+str(args.nt)+'_Nx'+str(args.Nx)+'_Re'+str(args.Re)+'_vode.pkl', 'wb') as output:
    pickle.dump(data_dict, output, pickle.HIGHEST_PROTOCOL)

In [ ]:
###########################################################
# First plot the solution on the original grid
###########################################################
fig = plt.figure(figsize=(18, 4), facecolor='white')
ax_u = fig.add_subplot(131)
ax_u_analy = fig.add_subplot(132)
ax_u_diff = fig.add_subplot(133)

X, T = np.meshgrid(grid_obj.x_grid, t.numpy())

ax_u.cla()
ax_u.set_title('Numerical Solution')
ax_u.set_xlabel('x')
ax_u.set_ylabel('t')
plot = ax_u.contourf(X, T, tf.squeeze(u, axis=1).numpy(), cmap=cm.coolwarm,
                           antialiased=False, levels=np.linspace(0, 0.5, 20), extend='min')
ax_u.set_xlim(grid_obj.x_grid[0], grid_obj.x_grid[-1])
ax_u.set_ylim(t[0], t[-1])
plt.colorbar(plot, ax=ax_u, shrink=0.5, aspect=10)

###########################################################
# Plot analytical solution
###########################################################
ad_eqn_ana_inst = ad_eqn_analy(grid_obj.x_grid, args)

u_analy = []

for i in range(args.nt):
    u_analy.append(tf.expand_dims(ad_eqn_ana_inst(t[i]), axis = 1))
    
u_analy = tf.concat(u_analy, axis=0)

ax_u_analy.cla()
ax_u_analy.set_title('Analytical Solution')
ax_u_analy.set_xlabel('x')
ax_u_analy.set_ylabel('t')
plot = ax_u_analy.contourf(X, T, tf.squeeze(u_analy, axis=1).numpy(), cmap=cm.coolwarm,
                           antialiased=False, levels=np.linspace(0, 0.5, 20), extend='min')
ax_u_analy.set_xlim(grid_obj.x_grid[0], grid_obj.x_grid[-1])
ax_u_analy.set_ylim(t[0], t[-1])
plt.colorbar(plot, ax=ax_u_analy, shrink=0.5, aspect=10)

###########################################################
# Plot difference w.r.t. analytical solution
###########################################################

ax_u_diff.cla()
ax_u_diff.set_title('Diff. with Analytical Solution')
ax_u_diff.set_xlabel('x')
ax_u_diff.set_ylabel('t')
plot = ax_u_diff.contourf(X, T, tf.squeeze(tf.abs(u - u_analy), axis=1).numpy(), cmap=cm.coolwarm,
                           antialiased=False, levels=np.linspace(0, 0.25, 20))
ax_u_diff.set_xlim(grid_obj.x_grid[0], grid_obj.x_grid[-1])
ax_u_diff.set_ylim(t[0], t[-1])
plt.colorbar(plot, ax=ax_u_diff, shrink=0.5, aspect=10)

fig.show()

In [ ]:
plt.plot(grid_obj.x_grid, u[0, 0, :])

In [ ]:
plt.plot(grid_obj.x_grid, np.einsum('ab, cb -> ca', sol_obj.full_vander_dx, u[:, 0, :].numpy())[0, ])

In [ ]:
plt.plot(grid_obj.x_grid, np.einsum('ab, cb -> ca', sol_obj.full_vander_dxx, u[:, 0, :].numpy())[0, ], 
        grid_obj.x_grid, np.einsum('ab, cb -> ca', sol_obj.deriv_obj.vander(grid_obj.x_grid, m=2, acc=4), u[:, 0, :].numpy())[0, ])

In [ ]:
fig, ax = plt.subplots()
line, = plt.plot([], [], 'o-', lw=2)
labels = ['Numerical', 'Analytical']

plotlays, plotcols = [3], ["blue","red"]
lines = []
for index in range(2):
    lobj = ax.plot([],[], 'o-', lw=2,color=plotcols[index])[0]
    lines.append(lobj)

def init():
    ax.set_xlim(0, 1.)
    ax.set_ylim(0, 0.5)
    ax.set_xlabel('x')
    ax.set_ylabel('u')
    
    for line in lines:
        line.set_data([],[])
    return lines

def update(frame):
    xdata = [grid_obj.x_grid, grid_obj.x_grid]
    ydata = [tf.squeeze(u, axis=1).numpy()[frame, :], tf.squeeze(u_analy, axis=1).numpy()[frame, :]]
    
    for lnum,line in enumerate(lines):
        line.set_data(xdata[lnum], ydata[lnum]) # set data for each line separately.
        line.set_label(labels[lnum])
        ax.legend(loc="upper right")

        fig.gca().autoscale_view()
        fig.gca().relim()
    return lines 

ani = FuncAnimation(fig, update, frames=np.arange(len(t.numpy())),
                    init_func=init, blit=True, interval=60)

ani.save('filename_res_Re'+str(args.Re)+'_Nx'+str(args.Nx)+'.gif', writer=PillowWriter(fps=24))